# 3 Reading and Wrting Data with Python

In [5]:
nobel_winners = [
    {
        'category': 'Physics',
        'name': 'Albert Einstein',
        'nationality': 'Swiss',
        'sex': 'male',
        'year': 1921
    },
    {
        'category': 'Physics',
        'name': 'Paul Dirac',
        'nationality': 'British',
        'sex': 'male',
        'year': 1933
    },
    {
        'category': 'Chemistry',
        'name': 'Marie Curie',
        'nationality': 'Polish',
        'sex': 'female',
        'year': 1911
    }
]

In [3]:
cols = nobel_winners[0].keys()
cols = sorted(cols)

In [4]:
with open('data/nobel_winners.csv', 'w') as f:
    f.write(','.join(cols) + '\n')
    
    for o in nobel_winners:
        row = [str(o[col]) for col in cols]
        f.write(','.join(row) + '\n')

In [10]:
with open('data/nobel_winners.csv') as f:
    for line in f.readlines():
        print(line)

category,name,nationality,sex,year

Pysics,Albert Einstein,Swiss,male,1921

Pysics,Paul Dirac,British,male,1933

Chemistry,Marie Curie,Polish,female,1911



In [9]:
import csv

with open('data/nobel_winners.csv', 'w') as f:
    fieldnames = nobel_winners[0].keys()
    fieldnames = sorted(fieldnames)
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for w in nobel_winners:
        writer.writerow(w)

In [11]:
with open('data/nobel_winners.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['category', 'name', 'nationality', 'sex', 'year']
['Pysics', 'Albert Einstein', 'Swiss', 'male', '1921']
['Pysics', 'Paul Dirac', 'British', 'male', '1933']
['Chemistry', 'Marie Curie', 'Polish', 'female', '1911']


In [12]:
with open('data/nobel_winners.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row)

OrderedDict([('category', 'Pysics'), ('name', 'Albert Einstein'), ('nationality', 'Swiss'), ('sex', 'male'), ('year', '1921')])
OrderedDict([('category', 'Pysics'), ('name', 'Paul Dirac'), ('nationality', 'British'), ('sex', 'male'), ('year', '1933')])
OrderedDict([('category', 'Chemistry'), ('name', 'Marie Curie'), ('nationality', 'Polish'), ('sex', 'female'), ('year', '1911')])


## JSON

In [13]:
import json

with open('data/nobel_winners.json', 'w') as f:
    json.dump(nobel_winners, f)

In [15]:
open('data/nobel_winners.json').read()

'[{"category": "Pysics", "name": "Albert Einstein", "nationality": "Swiss", "sex": "male", "year": 1921}, {"category": "Pysics", "name": "Paul Dirac", "nationality": "British", "sex": "male", "year": 1933}, {"category": "Chemistry", "name": "Marie Curie", "nationality": "Polish", "sex": "female", "year": 1911}]'

In [16]:
with open('data/nobel_winners.json') as f:
    nobel_winners = json.load(f)

In [17]:
nobel_winners

[{'category': 'Pysics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Pysics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

### Dealing with Dates and Times

In [22]:
import datetime
from dateutil import parser
import json

class JSONDateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.date, datetime.datetime)):
            return obj.isoformat()
        else:
            return json.JSONEncoder.default(self, obj)
        
def dumps(obj):
    return json.dumps(obj, cls=JSONDateTimeEncoder)

In [24]:
now_str = dumps({'time': datetime.datetime.now()})
now_str

'{"time": "2019-10-02T13:31:49.963370"}'

In [27]:
time_str = '2012/01/01 12:32:11'
dt = datetime.datetime.strptime(time_str, '%Y/%m/%d %H:%M:%S')
dt

datetime.datetime(2012, 1, 1, 12, 32, 11)

## SQL

In [29]:
from sqlalchemy import create_engine

engine = create_engine(
    'sqlite:///data/nobel_prize.db', echo=True)

In [41]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [61]:
Base.metadata.clear()

In [62]:
from sqlalchemy import Column, Integer, String, Enum

class Winner(Base):
    __tablename__ = 'Winner'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)    
    year = Column(Integer)
    nationality = Column(String)    
    sex = Column(Enum('male', 'female'))
    
    def __repr__(self):
        return f'<Winner(name={self.name}, category={self.category}, year={self.year})>'
    

In [75]:
Base.metadata.drop_all(engine)

2019-10-02 14:04:05,923 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Winner")
2019-10-02 14:04:05,924 INFO sqlalchemy.engine.base.Engine ()
2019-10-02 14:04:05,926 INFO sqlalchemy.engine.base.Engine 
DROP TABLE "Winner"
2019-10-02 14:04:05,927 INFO sqlalchemy.engine.base.Engine ()
2019-10-02 14:04:06,001 INFO sqlalchemy.engine.base.Engine COMMIT


In [76]:
Base.metadata.create_all(engine)

2019-10-02 14:04:19,635 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Winner")
2019-10-02 14:04:19,637 INFO sqlalchemy.engine.base.Engine ()
2019-10-02 14:04:19,638 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Winner")
2019-10-02 14:04:19,639 INFO sqlalchemy.engine.base.Engine ()
2019-10-02 14:04:19,641 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Winner" (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	category VARCHAR, 
	year INTEGER, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2019-10-02 14:04:19,642 INFO sqlalchemy.engine.base.Engine ()
2019-10-02 14:04:19,709 INFO sqlalchemy.engine.base.Engine COMMIT


In [77]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [65]:
albert = Winner(**nobel_winners[0])
session.add(albert)
session.new

IdentitySet([<Winner(name=Albert Einstein, category=Pysics, year=1921)>])

In [66]:
session.expunge(albert)
session.new

IdentitySet([])

In [78]:
winner_rows = [Winner(**w) for w in nobel_winners]
session.add_all(winner_rows)
session.commit()

2019-10-02 14:04:25,580 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-02 14:04:25,583 INFO sqlalchemy.engine.base.Engine INSERT INTO "Winner" (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2019-10-02 14:04:25,583 INFO sqlalchemy.engine.base.Engine ('Albert Einstein', 'Physics', 1921, 'Swiss', 'male')
2019-10-02 14:04:25,592 INFO sqlalchemy.engine.base.Engine INSERT INTO "Winner" (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2019-10-02 14:04:25,593 INFO sqlalchemy.engine.base.Engine ('Paul Dirac', 'Physics', 1933, 'British', 'male')
2019-10-02 14:04:25,594 INFO sqlalchemy.engine.base.Engine INSERT INTO "Winner" (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2019-10-02 14:04:25,595 INFO sqlalchemy.engine.base.Engine ('Marie Curie', 'Chemistry', 1911, 'Polish', 'female')
2019-10-02 14:04:25,596 INFO sqlalchemy.engine.base.Engine COMMIT


In [79]:
session.query(Winner).count()

2019-10-02 14:04:27,352 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-02 14:04:27,355 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner") AS anon_1
2019-10-02 14:04:27,357 INFO sqlalchemy.engine.base.Engine ()


3

In [80]:
result = session.query(Winner).filter_by(nationality='Swiss')
list(result)

2019-10-02 14:04:28,045 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" 
WHERE "Winner".nationality = ?
2019-10-02 14:04:28,047 INFO sqlalchemy.engine.base.Engine ('Swiss',)


[<Winner(name=Albert Einstein, category=Physics, year=1921)>]

In [81]:
result = session.query(Winner).filter(
    Winner.category == 'Physics',
    Winner.nationality != 'Swiss'
)
list(result)

2019-10-02 14:04:29,128 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" 
WHERE "Winner".category = ? AND "Winner".nationality != ?
2019-10-02 14:04:29,130 INFO sqlalchemy.engine.base.Engine ('Physics', 'Swiss')


[<Winner(name=Paul Dirac, category=Physics, year=1933)>]

In [82]:
session.query(Winner).get(3)

2019-10-02 14:04:44,319 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" 
WHERE "Winner".id = ?
2019-10-02 14:04:44,320 INFO sqlalchemy.engine.base.Engine (3,)


<Winner(name=Marie Curie, category=Chemistry, year=1911)>

In [83]:
res = session.query(Winner).order_by('year')
list(res)

2019-10-02 14:05:04,426 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" ORDER BY "Winner".year
2019-10-02 14:05:04,428 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name=Marie Curie, category=Chemistry, year=1911)>,
 <Winner(name=Albert Einstein, category=Physics, year=1921)>,
 <Winner(name=Paul Dirac, category=Physics, year=1933)>]

In [84]:
def inst_to_dict(inst, delete_id=True):
    dat = {}
    for column in inst.__table__.columns:
        dat[column.name] = getattr(inst, column.name)
    if delete_id:
        dat.pop('id')
    return dat

In [85]:
winner_rows = session.query(Winner)
nobel_winners = [inst_to_dict(w) for w in winner_rows]
nobel_winners

2019-10-02 14:18:29,249 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner"
2019-10-02 14:18:29,250 INFO sqlalchemy.engine.base.Engine ()


[{'name': 'Albert Einstein',
  'category': 'Physics',
  'year': 1921,
  'nationality': 'Swiss',
  'sex': 'male'},
 {'name': 'Paul Dirac',
  'category': 'Physics',
  'year': 1933,
  'nationality': 'British',
  'sex': 'male'},
 {'name': 'Marie Curie',
  'category': 'Chemistry',
  'year': 1911,
  'nationality': 'Polish',
  'sex': 'female'}]

In [86]:
marie = session.query(Winner).get(3)
marie.nationality = 'French'
session.dirty

IdentitySet([<Winner(name=Marie Curie, category=Chemistry, year=1911)>])

In [87]:
session.commit()

2019-10-02 14:19:32,109 INFO sqlalchemy.engine.base.Engine UPDATE "Winner" SET nationality=? WHERE "Winner".id = ?
2019-10-02 14:19:32,111 INFO sqlalchemy.engine.base.Engine ('French', 3)
2019-10-02 14:19:32,113 INFO sqlalchemy.engine.base.Engine COMMIT


In [89]:
session.dirty

IdentitySet([])

In [90]:
session.query(Winner).get(3).nationality

2019-10-02 14:19:55,393 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-10-02 14:19:55,395 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" 
WHERE "Winner".id = ?
2019-10-02 14:19:55,396 INFO sqlalchemy.engine.base.Engine (3,)


'French'

In [91]:
session.query(Winner).filter_by(name='Albert Einstein').delete()

2019-10-02 14:20:25,750 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" 
WHERE "Winner".id = ?
2019-10-02 14:20:25,751 INFO sqlalchemy.engine.base.Engine (1,)
2019-10-02 14:20:25,754 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner" 
WHERE "Winner".id = ?
2019-10-02 14:20:25,754 INFO sqlalchemy.engine.base.Engine (2,)
2019-10-02 14:20:25,756 INFO sqlalchemy.engine.base.Engine DELETE FROM "Winner" WHERE "Winner".name = ?
2019-10-02 14:20:25,756 INFO sqlalchemy.engine.base.Engine ('Albert Einstein',)


1

In [92]:
list(session.query(Winner))

2019-10-02 14:20:36,301 INFO sqlalchemy.engine.base.Engine SELECT "Winner".id AS "Winner_id", "Winner".name AS "Winner_name", "Winner".category AS "Winner_category", "Winner".year AS "Winner_year", "Winner".nationality AS "Winner_nationality", "Winner".sex AS "Winner_sex" 
FROM "Winner"
2019-10-02 14:20:36,303 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name=Paul Dirac, category=Physics, year=1933)>,
 <Winner(name=Marie Curie, category=Chemistry, year=1911)>]

In [94]:
Winner.__table__.drop(engine)

2019-10-02 14:22:40,801 INFO sqlalchemy.engine.base.Engine 
DROP TABLE "Winner"
2019-10-02 14:22:40,802 INFO sqlalchemy.engine.base.Engine ()
2019-10-02 14:22:45,811 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
DROP TABLE "Winner"]
(Background on this error at: http://sqlalche.me/e/e3q8)

### Easier SQL with Dataset

In [1]:
import dataset

db = dataset.connect('sqlite:///data/nobel_prize.db')

In [2]:
wtable = db['winners']
winners = wtable.find()
winners = list(winners)
winners

[OrderedDict([('id', 1),
              ('name', 'Albert Einstein'),
              ('category', 'Pysics'),
              ('year', 1921),
              ('nationality', 'Swiss'),
              ('sex', 'male')]),
 OrderedDict([('id', 2),
              ('name', 'Paul Dirac'),
              ('category', 'Pysics'),
              ('year', 1933),
              ('nationality', 'British'),
              ('sex', 'male')]),
 OrderedDict([('id', 3),
              ('name', 'Marie Curie'),
              ('category', 'Chemistry'),
              ('year', 1911),
              ('nationality', 'Polish'),
              ('sex', 'female')])]

In [3]:
wtable = db['winners']
wtable.drop()

In [4]:
wtable = db['winners']
wtable.find()

In [6]:
with db as tx:
    for w in nobel_winners:
        tx['winners'].insert(w)

/home/asurin/anaconda3/envs/pyjsviz/lib/python3.7/site-packages/dataset/table.py:218: RuntimeWarning: Changing the database schema inside a transaction in a multi-threaded environment is likely to lead to race conditions and synchronization issues.
  RuntimeWarning)


In [7]:
list(db['winners'].find())

[OrderedDict([('id', 1),
              ('category', 'Physics'),
              ('name', 'Albert Einstein'),
              ('nationality', 'Swiss'),
              ('sex', 'male'),
              ('year', 1921)]),
 OrderedDict([('id', 2),
              ('category', 'Physics'),
              ('name', 'Paul Dirac'),
              ('nationality', 'British'),
              ('sex', 'male'),
              ('year', 1933)]),
 OrderedDict([('id', 3),
              ('category', 'Chemistry'),
              ('name', 'Marie Curie'),
              ('nationality', 'Polish'),
              ('sex', 'female'),
              ('year', 1911)])]

In [18]:
import datafreeze
winners = db['winners'].find()
datafreeze.freeze(winners, format='csv',
               filename='data/nobel_winners_ds.csv')
print(open('data/nobel_winners_ds.csv').read())

id,category,name,nationality,sex,year
1,Physics,Albert Einstein,Swiss,male,1921
2,Physics,Paul Dirac,British,male,1933
3,Chemistry,Marie Curie,Polish,female,1911



## MongoDB

In [21]:
from pymongo import MongoClient

client = MongoClient()
db = client.nobel_prize
coll = db.winners

In [25]:
DB_NOBEL_PRIZE = 'nobel_prize'
COLL_WINNERS = 'winners'

In [22]:
from pymongo import MongoClient

def get_mongo_database(db_name, host='pi.hole',
                       port=27017, username=None, password=None):
    """ Get named database from MongoDB with/out authentication """
    if username and password:
        mongo_url = f'mongodb://{username}:{password}@{host}/{db_name}'
        conn = MongoClient(mongo_url)
    else:
        conn = MongoClient(host, port)
        
    return conn[db_name]

In [26]:
db = get_mongo_database(DB_NOBEL_PRIZE)
coll = db[COLL_WINNERS]

In [27]:
coll.insert(nobel_winners)

/home/asurin/anaconda3/envs/pyjsviz/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5d94ffe70138384d76036f82'),
 ObjectId('5d94ffe70138384d76036f83'),
 ObjectId('5d94ffe70138384d76036f84')]

In [28]:
nobel_winners

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921,
  '_id': ObjectId('5d94ffe70138384d76036f82')},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933,
  '_id': ObjectId('5d94ffe70138384d76036f83')},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911,
  '_id': ObjectId('5d94ffe70138384d76036f84')}]

In [31]:
import bson
oid = bson.ObjectId()
oid.generation_time

datetime.datetime(2019, 10, 2, 19, 53, 23, tzinfo=<bson.tz_util.FixedOffset object at 0x7f2e0362ce10>)

In [32]:
res = coll.find({'category': 'Chemistry'})
list(res)

[{'_id': ObjectId('5d94ffe70138384d76036f84'),
  'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [33]:
res = coll.find({'year': {'$gt': 1930}})
list(res)

[{'_id': ObjectId('5d94ffe70138384d76036f83'),
  'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933}]

In [34]:
res = coll.find({'$or': [{'year': {'$gt': 1930}}, 
                         {'sex': 'female'}]})
list(res)

[{'_id': ObjectId('5d94ffe70138384d76036f83'),
  'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'_id': ObjectId('5d94ffe70138384d76036f84'),
  'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

In [35]:
def mongo_coll_to_dicts(dbname='test', collname='test',
                        query={}, del_id=True, **kw):
    db = get_mongo_database(dbname, **kw)
    res = list(db[collname].find(query))
    
    if del_id:
        for r in res:
            r.pop('_id')
    return res

In [36]:
mongo_coll_to_dicts(DB_NOBEL_PRIZE, COLL_WINNERS)

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]

## Dealing with Dates, Times, and Complex Data

In [37]:
from datetime import datetime

d = datetime.now()
d.isoformat()

'2019-10-02T14:59:26.461768'